In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import jax.numpy as jnp
from jax import random

import numpyro 
from numpyro.infer.autoguide import AutoLaplaceApproximation
from numpyro.infer import SVI, Trace_ELBO
from numpyro.diagnostics import print_summary
import numpyro.distributions as dist
import numpyro.optim as optim
from sklearn.preprocessing import StandardScaler

In [2]:
filename = '../rethinking/data/foxes.csv'

In [3]:
df = pd.read_csv(filename, sep=';')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116 entries, 0 to 115
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   group      116 non-null    int64  
 1   avgfood    116 non-null    float64
 2   groupsize  116 non-null    int64  
 3   area       116 non-null    float64
 4   weight     116 non-null    float64
dtypes: float64(3), int64(2)
memory usage: 4.7 KB


In [5]:
df.head()

,group,avgfood,groupsize,area,weight
0,1,0.37,2,1.09,5.02
1,1,0.37,2,1.09,2.84
2,2,0.53,2,2.05,5.33
3,2,0.53,2,2.05,6.07
4,3,0.49,2,2.12,5.85


In [6]:
df['avgfood'] = (df['avgfood'] - np.mean(df['avgfood'])) / np.std(df['avgfood'])
df['groupsize'] = (df['groupsize'] - np.mean(df['groupsize'])) / np.std(df['groupsize'])
df['area'] = (df['area'] - np.mean(df['area'])) / np.std(df['area'])
df['weight'] = (df['weight'] - np.mean(df['weight'])) / np.std(df['weight'])


F is `avgfood`, G is `groupsize`, A is `area`, and W is `weight`.

# Question 1

Simulate the effect of A ("area") on F ("avgfood")

Note: Numpyro is severly lacking in documentation but how to get it up and running was inferred from examples, such as this: https://fehiepsi.github.io/rethinking-numpyro/05-the-many-variables-and-the-spurious-waffles.html

In [27]:
def model(A, D=None):
    
    a = numpyro.sample("a", dist.Normal(0, 0.2))
    bA = numpyro.sample("bA", dist.Normal(0, 0.5))
    mu = a + bA * A
    sigma = numpyro.sample("sigma", dist.Exponential(1))
    F = numpyro.sample("F", dist.Normal(mu, sigma), obs=D)

pls_document_your_framework = AutoLaplaceApproximation(model)

svi = SVI(
    model, 
    pls_document_your_framework, 
    optim.Adam(1), 
    Trace_ELBO(), 
    A = df['area'].values,
    D = df['avgfood'].values
)

In [33]:
result = svi.run(random.PRNGKey(0), 10000)
result.params

100%|█| 10000/10000 [00:02<00:00, 4593.86it/s, init loss: 808.0281, avg. loss [9


{'auto_loc': DeviceArray([ 0.01509295,  0.9012608 , -0.73298544], dtype=float32)}

In [34]:
posterior = pls_document_your_framework.sample_posterior(
    random.PRNGKey(1), 
    result.params, 
    (10000,)
)

In [37]:
print_summary(posterior, group_by_chain=False)


                mean       std    median      5.0%     95.0%     n_eff     r_hat
         a      0.01      0.04      0.02     -0.06      0.08   9971.53      1.00
        bA      0.90      0.04      0.90      0.83      0.97  10073.39      1.00
     sigma      0.48      0.03      0.48      0.43      0.53   9782.38      1.00



# Question 2

Infer both the total and direct causal effects of adding food F to a territory on the weight W of foxes. Which covariates do you need to adjust for in each case? In light of your estimates from this problem and the previous one, what do you think is going on with these foxes? 

In [38]:
def model_one(F, D=None):
    a = numpyro.sample("a", dist.Normal(0, 0.2))
    bF = numpyro.sample("bF", dist.Normal(0, 0.5))
    sigma = numpyro.sample("sigma", dist.Exponential(1))
    mu = a + bF * F
    W = numpyro.sample("W", dist.Normal(mu, sigma), obs=D)
    
autoL = AutoLaplaceApproximation(model_one)
svi = SVI(
    model_one, 
    autoL, 
    optim.Adam(1), 
    Trace_ELBO(), 
    F=df['avgfood'].values,
    D=df['weight'].values
)

In [40]:
result = svi.run(random.PRNGKey(0), 10000)
result.params

100%|█| 10000/10000 [00:02<00:00, 4625.08it/s, init loss: 3411.7129, avg. loss [


{'auto_loc': DeviceArray([-0.00094191, -0.0351629 , -0.01127534], dtype=float32)}

In [43]:
posterior = autoL.sample_posterior(random.PRNGKey(1), result.params, (10000,))
print_summary(posterior, group_by_chain=False)


                mean       std    median      5.0%     95.0%     n_eff     r_hat
         a     -0.00      0.08     -0.00     -0.14      0.13   9971.53      1.00
        bF     -0.04      0.09     -0.04     -0.18      0.11  10073.69      1.00
     sigma      0.99      0.06      0.99      0.88      1.09   9814.02      1.00



In [47]:
def model_two(F, G, D=None):
    a = numpyro.sample("a", dist.Normal(0, 0.2))
    bF = numpyro.sample("bF", dist.Normal(0, 0.5))
    bG = numpyro.sample("bG", dist.Normal(0, 0.5))
    sigma = numpyro.sample("sigma", dist.Exponential(1))
    mu = a + bF * F + bG*G
    W = numpyro.sample("W", dist.Normal(mu, sigma), obs=D)
    
autoL = AutoLaplaceApproximation(model_two)
svi = SVI(
    model_two, 
    autoL, 
    optim.Adam(1), 
    Trace_ELBO(), 
    F=df['avgfood'].values,
    G=df['groupsize'].values,
    D=df['weight'].values
)

In [48]:
result = svi.run(random.PRNGKey(0), 10000)
result.params

100%|█| 10000/10000 [00:02<00:00, 4523.28it/s, init loss: 1578.7650, avg. loss [


{'auto_loc': DeviceArray([-0.01131386,  0.47927392, -0.569963  , -0.04578316], dtype=float32)}

In [49]:
posterior = autoL.sample_posterior(random.PRNGKey(1), result.params, (10000,))
print_summary(posterior, group_by_chain=False)


                mean       std    median      5.0%     95.0%     n_eff     r_hat
         a     -0.01      0.08     -0.01     -0.14      0.13   9990.20      1.00
        bF      0.48      0.18      0.48      0.18      0.77   9233.25      1.00
        bG     -0.57      0.18     -0.57     -0.88     -0.28   9864.12      1.00
     sigma      0.96      0.06      0.96      0.85      1.06   9988.26      1.00



In [52]:
def model_three(F, D=None):
    a = numpyro.sample("a", dist.Normal(0, 0.2))
    bF = numpyro.sample("bF", dist.Normal(0, 0.5))
    sigma = numpyro.sample("sigma", dist.Exponential(1))
    
    mu = a + bF * F
    G = numpyro.sample("W", dist.Normal(mu, sigma), obs=D)
    
autoL = AutoLaplaceApproximation(model_three)
svi = SVI(
    model_three, 
    autoL, 
    optim.Adam(1), 
    Trace_ELBO(), 
    F=df['avgfood'].values,
    D=df['groupsize'].values,
)

result = svi.run(random.PRNGKey(0), 10000)
posterior = autoL.sample_posterior(random.PRNGKey(1), result.params, (10000,))
print_summary(posterior, group_by_chain=False)

100%|█| 10000/10000 [00:02<00:00, 4606.71it/s, init loss: 755.3344, avg. loss [9



                mean       std    median      5.0%     95.0%     n_eff     r_hat
         a     -0.05      0.04     -0.05     -0.12      0.01   9971.53      1.00
        bF      0.87      0.04      0.87      0.81      0.94  10071.79      1.00
     sigma      0.43      0.03      0.43      0.38      0.48   9902.39      1.00

